Import all libraries here

In [ ]:
from typing import List, NewType, Set, Tuple, Iterable
from pprint import pprint
from operator import mul
from functools import reduce
import numpy as np


Set the grid length

In [ ]:
grid_length = 10
# grid_length = 5


We need a method to quickly create a grid

In [ ]:
def initialise_square_grid(length: int) -> np.ndarray:
    return np.zeros((length, length), dtype=int)


Create types for sanity checking

In [ ]:
RowIndex = NewType("RowIndex", int)
ColIndex = NewType("ColIndex", int)
Element = NewType("Element", int)  # Refers to a number in our grid
# Index of the component in the components list
ComponentID = NewType("ComponentID", int)
ComponentIndex = NewType("ComponentIndex", int)


Initialise our grid

In [ ]:
grid: np.ndarray = initialise_square_grid(grid_length)


Put pre-existing numbers in a list

In [ ]:
initial_numbers: List[Tuple[RowIndex, ColIndex, Element]] = [
    (0, 1, 3),
    (0, 5, 7),
    (1, 3, 4),
    (2, 8, 2),
    (3, 3, 1),
    (4, 0, 6),
    (4, 2, 1),
    (5, 7, 3),
    (5, 9, 6),
    (6, 6, 2),
    (7, 1, 2),
    (8, 6, 6),
    (9, 4, 5),
    (9, 8, 2)
]
# initial_numbers: List[Tuple[RowIndex, ColIndex, Element]] = [
#     (1, 4, 2),
#     (2, 2, 4),
#     (3, 0, 3)
# ]


Put components in a list

In [ ]:
components: List[List[Tuple[RowIndex, ColIndex]]] = [
    [(0, 0), (1, 0), (1, 1), (2, 0), (2, 1),
     (3, 0), (3, 1), (4, 0), (5, 0), (6, 0)],
    [(5, 1)],
    [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (1, 4)],
    [(0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 5), (1, 8), (1, 9)],
    [(1, 6), (1, 7), (2, 7)],
    [(2, 4), (2, 5)],
    [(2, 2), (2, 3), (3, 3)],
    [(3, 2), (4, 1), (4, 2), (5, 2)],
    [(6, 1), (7, 0), (7, 1), (8, 0), (8, 1),
     (8, 2), (9, 0), (9, 1), (9, 2), (9, 3)],
    [(2, 6), (3, 5), (3, 6), (3, 7), (4, 7), (4, 8)],
    [(3, 4), (4, 3), (4, 4)],
    [(4, 5), (5, 5)],
    [(4, 6)],
    [(5, 3), (6, 3), (6, 4), (6, 5), (7, 4)],
    [(5, 4)],
    [(5, 6), (5, 7), (6, 6)],
    [(6, 2), (7, 2)],
    [(6, 7), (6, 8), (6, 9)],
    [(2, 8), (2, 9), (3, 8), (3, 9), (4, 9), (5, 8), (5, 9)],
    [(7, 6), (7, 9), (8, 5), (8, 6), (8, 9), (9, 6), (9, 7), (9, 8), (9, 9)],
    [(7, 7), (7, 8), (8, 7), (8, 8)],
    [(7, 3), (8, 3), (8, 4), (9, 4), (9, 5)],
    [(7, 5)]
]

# components: List[List[Tuple[RowIndex, ColIndex]]] = [
#     [(0, 0), (0, 1), (1, 1), (1, 2), (2, 2)],
#     [(2, 1)],
#     [(1, 0), (2, 0), (3, 0), (4, 0), (4, 1)],
#     [(3, 1), (3, 2), (4, 2)],
#     [(4, 3)],
#     [(2, 3), (3, 3), (3, 4), (4, 4)],
#     [(1, 3)],
#     [(0, 2), (0, 3), (0, 4), (1, 4), (2, 4)]
# ]


In [ ]:
print(sum(map(lambda x: len(x), components)))


Perform sanity check on our components

In [ ]:
tuple_set: Set[Tuple[RowIndex, ColIndex]] = {
    tup for component in components for tup in component}

for i in range(grid_length):
    for j in range(grid_length):
        if (i, j) not in tuple_set:
            raise Exception((i, j), "not accounted for")


Map all cells to their component within the component list

In [ ]:
cell_index: np.ndarray = initialise_square_grid(grid_length)

for component_id in range(len(components)):
    for row, col in components[component_id]:
        cell_index[row][col] = component_id


We need a function to get the component id of a cell

In [ ]:

def get_component_id(row: RowIndex, col: ColIndex) -> int:
    """
    Returns the component number of this cell.
    """
    return cell_index[row][col]


For each component, we will need to track which numbers we have already inserted

In [ ]:
numbers_used: np.ndarray = np.zeros(len(components), dtype=int)

We need a function to check if a particular component needs a number n, that is, if n is less than or equal to the size of the component and no cell in that component currently has n

In [ ]:
def needs(component: ComponentID, num: int) -> bool:
    return num <= len(components[component]) \
        and (numbers_used[component] & (1 << num)) == 0


Write a function to get all cells that are exactly distance k away from a given cell

In [ ]:
def distance_k_away(grid: np.ndarray, row: RowIndex, col: ColIndex, k: int) -> Iterable[Tuple[RowIndex, ColIndex]]:
    for possible_row in range(max(0, row - k), min(len(grid) - 1, row + k) + 1):
        column_offset: int = k - abs(row - possible_row)
        possible_columns = [col + column_offset, col - column_offset]
        for column in possible_columns:
            if 0 <= column < len(grid[0]):
                yield possible_row, column

            if column_offset == 0:
                break



We also need a function to check if there's a cell with a particular number within distance less than k of a cell:

In [ ]:
def have_number_less_than_distance_k(grid: np.ndarray,
                                     row: RowIndex,
                                     col: ColIndex,
                                     desiredNumber: int,
                                     k: int) -> bool:
    for distance in range(1, k):  # [1, ..., k - 1]
        for r, c in distance_k_away(grid, row, col, distance):
            if grid[r][c] == desiredNumber:
                return True
    return False


We need a function to insert a number into a grid. It should update the grid, and update `numbers_used`. We also need the undo-er for this function

In [ ]:
def insert(grid: np.ndarray, row: RowIndex, col: ColIndex, num: int) -> None:
    grid[row][col] = num
    numbers_used[get_component_id(row, col)] |= (1 << num)


def remove(grid: np.ndarray, row: RowIndex, col: ColIndex) -> None:
    num: int = grid[row][col]
    numbers_used[get_component_id(row, col)] &= ~(1 << num)
    grid[row][col] = 0


Insert pre-existing numbers into the grid and the `numbers_used` map

In [ ]:
for row, column, element in initial_numbers:
    insert(grid, row, column, element)

pprint(grid)


We need to solve for as many cells as possible to ensure that backtracking does as little as possible

In [ ]:
def partial_solve(grid: np.ndarray):
    while 1:
        made_progress: bool = False

        # For all components with only one number not filled in, fill it in
        # and finish that component
        for component in range(len(components)):
            if bin(numbers_used[component]).count("1") == (len(components[component]) - 1):
                index = 0
                while (numbers_used[component] & (1 << index)) == 1:
                    index += 1

                for row, col in components[component]:
                    if grid[row][col] == 0:
                        # The + 1 is due to 0-indexing
                        insert(grid, row, col, index + 1)
                        made_progress = True
                        break

        # For all components c and unfilled numbers u, if u has only one spot
        # it can go into, fill that in
        for component in range(len(components)):
            for number in range(1, len(components[component]) + 1):
                if not (numbers_used[component] & (1 << number)):
                    # Get all unfilled squares
                    unfilled_squares = [(row, col) for (
                        row, col) in components[component] if grid[row][col] == 0]

                    # For each square, see how many squares might be a candidate
                    candidates = [(row, col) for (row, col) in unfilled_squares if not have_number_less_than_distance_k(
                        grid, row, col, number, number)]
                    if len(candidates) == 1:
                        # Only 1 square where this number can be filled
                        insert(grid, candidates[0][0],
                               candidates[0][1], number)
                        made_progress = True

        # For all currently filled numbers, if there is only one empty square that can be its nearest neighbour,
        # then that one empty square must contain that number
        for row in range(len(grid)):
            for col in range(len(grid[0])):
                if grid[row][col] > 0:
                    # Find all candidates for its nearest neighbours
                    neighbours: List[Tuple[RowIndex, ColIndex]
                                     ] = list(distance_k_away(grid, row, col, grid[row][col]))

                    # Remove all neighbours that are from the same component
                    neighbours = list(set(neighbours).difference(
                        set(components[get_component_id(row, col)])))

                    # If this cell already has a nearest neighbour, move on
                    if [(r, c) for (r, c) in neighbours if grid[r][c] == grid[row][col]]:
                        continue

                    candidates = [(r, c)
                                  for (r, c) in neighbours if grid[r][c] == 0]

                    if len(candidates) == 1:
                        insert(grid, candidates[0][0],
                               candidates[0][1], grid[row][col])
                        made_progress = True

        if not made_progress:
            break


In [ ]:
partial_solve(grid)
pprint(grid)

# solved = 0
# for row in grid:
#     for num in row:
#         if num > 0:
#             solved += 1

# print(solved)


Define a backtracking function to try inserting numbers into the grid using brute-force (with pruning)

In [ ]:
def backtrack(grid: np.ndarray,
              component_id: ComponentID = 0,
              component_index: ComponentIndex = 0) -> bool:
    # If you've reached past the last component, you've succeeded
    if component_id == len(components):
        return True

    # If you've reached the end of a component, go to the next component
    if component_index == len(components[component_id]):
        return backtrack(grid, component_id + 1, 0)

    row, col = components[component_id][component_index]

    if grid[row][col] == 0:  # You need to try every possible number
        # The size of this component determines
        # the range of possible numbers
        size_of_component: int = len(components[component_id])

        for possible_num in range(1, size_of_component + 1):
            if not (numbers_used[component_id] & (1 << possible_num)) \
                    and not have_number_less_than_distance_k(grid, row, col, possible_num, possible_num):
                # Then try using this number
                insert(grid, row, col, possible_num)

                if backtrack(grid, component_id, component_index):
                    return True

                remove(grid, row, col)

        # If there is no number that you can put here, it's a lost cause
        return False

    # There's a number here, which means either it pre-existed, or some other number
    # gave you this number. We cannot just delete it if we can't succeed in filling up the
    # rest of the grid, we can only report the failure.

    this_number: Element = grid[row][col]

    # Get all cells that are this_number away
    neighbours: List[Tuple[RowIndex, ColIndex]
                     ] = list(distance_k_away(grid, row, col, this_number))

    # If any cells have this_number, we don't need to create any more of this_number. Move on
    for r, c in neighbours:
        if grid[r][c] == this_number:
            return backtrack(grid, component_id, component_index + 1)

    # Try setting each of them to this_number, and see if you can complete the grid this way
    for row, col in neighbours:
        # Filter out all cells that have a number other than 0
        if grid[row][col] > 0 or \
                not needs(get_component_id(row, col), this_number) or \
                have_number_less_than_distance_k(grid, row, col, this_number, this_number):
            continue

        insert(grid, row, col, this_number)

        # Go on with your life, see whether you succeed
        if backtrack(grid, component_id, component_index + 1):
            return True

        # Ok you failed, but you still have more neighbours to try
        remove(grid, row, col)

    # If you've tried all neighbours, and nobody can be this_number away from you, it's a lost cause
    return False


Try solving the grid:

In [ ]:
backtrack(grid)


Let's see what we got!

In [ ]:
pprint(grid)


Now let's write a function to multiply numbers along a row and sum up the products across rows:

In [ ]:
def calculateGrid(grid: List[List[int]]) -> int:
    return sum(map(lambda x: reduce(mul, x), grid))


In [ ]:
calculateGrid(grid)
